In [1]:
import os
from pathlib import Path
from openai import OpenAI, AsyncOpenAI
from dotenv import load_dotenv
from datasets import load_dataset, Dataset

import nest_asyncio
nest_asyncio.apply()

In [2]:
dataset_name = load_dataset(os.getenv("HF_HUB_DATASET_NAME"))
assert dataset_name

In [3]:
# select the first 32 examples
sampled_dataset = dataset_name['train'].select(range(2))
len(sampled_dataset)

2

In [13]:
system_message = """
You are given 5 sentences from a story in a shuffled order. 
Each sentence has a serial number assoiciated with it. 
You can find the serial number at the beginning of each sentence, for example: 1. <sentence>.
Your task is to predict the correct order of sentences which would resemble the original story and then return the serial numbers as a comma separated string.

#IMPORTANT
There are exactly five sentences.
You should only return the serial numbers as a comma separated string.
"""

client = AsyncOpenAI(
    api_key="token-abc123",
    base_url=os.getenv("VLLM_ENDPOINT_URL")
)

In [8]:
from tqdm.auto import tqdm


for _, data in tqdm(enumerate(sampled_dataset)):
    formatted_inputs = []
    for idx, sent in enumerate(data["shuffled_sentences"]):
        formatted_inputs.append( f"{idx + 1}. {sent}")
    

0it [00:00, ?it/s]

In [14]:
response =  await client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": formatted_inputs[0]}
    ],
    temperature=0.0,
)

response

ChatCompletion(id='chat-5fdd157dd0364f3bbe5333132c599e0b', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Based on the given sentence, it appears that the story is about Carrie, a character who enjoys riding a bike. \n\nA possible order for the story could be:\n\n- 1. Carrie would sneak rides on her sister's bike.\n- 2. Her sister, who was a bit older, had just learned how to ride a bike herself.\n- 3. The sister was not too keen on Carrie using her bike, but Carrie's enthusiasm was hard to resist.\n- 4. One day, Carrie decided to take her sister's bike for a longer ride than usual.\n- 5. As she rode further and further away from home, Carrie began to feel a sense of freedom she had never experienced before.\n\nSo, the serial numbers in the correct order are: 1, 2, 3, 4, 5", role='assistant', function_call=None, tool_calls=[]), stop_reason=None)], created=1722345356, model='meta-llama/Meta-Llama-3.1-8B-Instruct', object='chat.comple

In [15]:
response.choices[0].message.content

"Based on the given sentence, it appears that the story is about Carrie, a character who enjoys riding a bike. \n\nA possible order for the story could be:\n\n- 1. Carrie would sneak rides on her sister's bike.\n- 2. Her sister, who was a bit older, had just learned how to ride a bike herself.\n- 3. The sister was not too keen on Carrie using her bike, but Carrie's enthusiasm was hard to resist.\n- 4. One day, Carrie decided to take her sister's bike for a longer ride than usual.\n- 5. As she rode further and further away from home, Carrie began to feel a sense of freedom she had never experienced before.\n\nSo, the serial numbers in the correct order are: 1, 2, 3, 4, 5"